<a href="https://colab.research.google.com/github/joseguambo1994/Awesome-CS-Books/blob/master/MasterUnirArchitectureDiagram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install diagrams
!apt-get install graphviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.1/25.1 MB 31.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
from diagrams import Diagram, Cluster, Edge
from diagrams.aws.compute import Lambda
from diagrams.aws.storage import S3
from diagrams.aws.ml import Rekognition
from diagrams.aws.devtools import Codecommit
from diagrams.aws.devtools import Codebuild
from diagrams.aws.devtools import Codedeploy
from diagrams.aws.management import Cloudwatch
from diagrams.aws.security import IAM
from diagrams.generic.device import Mobile  # User representation
from diagrams.custom import Custom  # For adding a custom image

graph_attr = {
    "splines": "spline"
}

# Path to your custom image in Colab
image_path = "/content/gappnaderov1.png"

with Diagram("Gappnadero AWS Architecture", show=True, graph_attr=graph_attr):

    # Auth & Monitoring Cluster
    with Cluster("Auth & Monitoring", graph_attr={"style": "filled", "fillcolor": "#E3F2FD"}):
        iam = IAM("IAM")
        monitoring = Cloudwatch("CloudWatch")

    # CI/CD Pipeline with a Light Pink Background
    with Cluster("CI/CD Pipeline", graph_attr={"style": "filled", "fillcolor": "#FFE6E6"}):
        repo = Codecommit("AWS CodeCommit\nRepo")
        build_main = Codebuild("AWS CodeBuild\nBuildProject")
        pipeline_main = Codedeploy("AWS CodeDeploy\nPipeline")
        build_staging = Codebuild("AWS CodeBuild\nBuildStaging")
        pipeline_staging = Codedeploy("AWS CodeDeploy\nStaging")

    # Apply Dotted Purple Arrows for CI/CD Connections
    repo >> Edge(style="dotted", color="purple") >> [build_main, build_staging]
    build_main >> Edge(style="dotted", color="purple") >> pipeline_main
    build_staging >> Edge(style="dotted", color="purple") >> pipeline_staging

    # S3 Buckets
    s3_raw = S3("Amazon S3\ncattle")
    s3_processed = S3("Amazon S3\ncattle-processed")

    # Lambda Functions
    with Cluster("Lambda Processing"):
        traffic_balancer = Lambda("Lambda\ntrafficBalancer")
        lambda_staging = Lambda("Lambda\nanalyze-image\n(staging)")
        lambda_production = Lambda("Lambda\nanalyze-image\n(production)")

    # Rekognition Service
    rekognition = Rekognition("AWS Rekognition")

    # User Representation
    user = Mobile("User\nGappnadero App")

    # Slightly Bigger Custom Image Representation
    analyzed_image = Custom("Analyzed Image", image_path, width="3.0", height="3.0")  # Increased size

    # CI/CD Deployment
    pipeline_main >> Edge(style="dotted", color="purple") >> lambda_production
    pipeline_staging >> Edge(style="dotted", color="purple") >> lambda_staging

    # Data Flow
    s3_raw >> traffic_balancer >> [lambda_staging, lambda_production]

    # Bidirectional Arrows Between AWS Rekognition and Lambda Functions
    rekognition << Edge(dir="both") >> lambda_staging
    rekognition << Edge(dir="both") >> lambda_production

    lambda_staging >> s3_processed
    lambda_production >> s3_processed
    s3_processed >> user  # User receives processed images

    # Connect Bigger Custom Image to Mobile User
    user >> analyzed_image


In [ ]:
from diagrams import Diagram, Cluster, Edge
from diagrams.azure.compute import FunctionApps
from diagrams.azure.storage import BlobStorage
from diagrams.azure.storage import QueuesStorage
from diagrams.azure.ml import CognitiveServices
from diagrams.azure.devops import Repos, Pipelines
from diagrams.azure.identity import ManagedIdentities
from diagrams.azure.devops import ApplicationInsights
from diagrams.generic.device import Mobile  # User representation
from diagrams.custom import Custom  # For adding a custom image

graph_attr = {
    "splines": "spline"
}

# Path to your custom image in Colab
image_path = "/content/gappnaderov1.png"

with Diagram("Gappnadero Azure Architecture", show=True, graph_attr=graph_attr):

    # Auth & Monitoring Cluster
    with Cluster("Auth & Monitoring", graph_attr={"style": "filled", "fillcolor": "#E3F2FD"}):
        identity = ManagedIdentities("Microsoft Entra ID")
        monitoring = ApplicationInsights("Application Insights")

    # CI/CD Pipeline with a Light Pink Background
    with Cluster("CI/CD Pipeline", graph_attr={"style": "filled", "fillcolor": "#FFE6E6"}):
        repo = Repos("Azure Repos\nGappnadero-Tesis-Unir")
        pipeline = Pipelines("Azure Pipelines\nGappnadero-Tesis-Unir")

    # Blob Storage
    blob_raw = BlobStorage("Blob Container\ncattle")
    blob_processed = BlobStorage("Blob Container\ncattle-processed")

    # Storage Queues (Rearranged for Clarity)
    with Cluster("Storage Queues", graph_attr={"style": "filled", "fillcolor": "#F0E68C"}):  # Light yellow for differentiation
        queue_production = QueuesStorage("Azure Storage Queue\nproduction-queue")
        queue_staging = QueuesStorage("Azure Storage Queue\nstaging-queue")

    # Azure Functions (Declared Only Once)
    with Cluster("Azure Function Processing"):
        traffic_balancer = FunctionApps("Azure Function\ntraffic-balancer")
        analyze_v1 = FunctionApps("Azure Function\nanalyze-image-azure-v1")
        analyze_v2 = FunctionApps("Azure Function\nanalyze-image-azure-v2")

    # Azure AI Services (Computer Vision)
    azure_ai = CognitiveServices("Azure AI Services\nComputer Vision Instance")

    # User Representation
    user = Mobile("User\nGappnadero App")

    # Custom Image Representation
    analyzed_image = Custom("Analyzed Image", image_path, width="3.0", height="3.0")

    # CI/CD Deployment (Now correctly references the existing functions)
    pipeline >> Edge(style="dotted", color="purple") >> [analyze_v1, analyze_v2]

    # Data Flow (Queues Positioned More Clearly)
    blob_raw >> traffic_balancer
    traffic_balancer >> queue_production >> analyze_v1
    traffic_balancer >> queue_staging >> analyze_v2

    # Bidirectional Arrows Between Azure AI and Functions
    azure_ai << Edge(dir="both") >> analyze_v1
    azure_ai << Edge(dir="both") >> analyze_v2

    analyze_v1 >> blob_processed
    analyze_v2 >> blob_processed
    blob_processed >> user  # User receives processed images

    # Connect Custom Image to Mobile User
    user >> analyzed_image
